In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras

from tqdm import tqdm_notebook, tqdm
from tensorflow.python.keras.engine.base_layer import Layer
from keras import backend as K
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation
from keras.optimizers import Adam
from tensorflow.python.keras import backend as K
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, LSTM
from sklearn.model_selection import train_test_split


In [ ]:
data = pd.read_csv("/content/drive/Shared drives/SK플래닛_OCB/raw_data/동신&용준/data_new.csv", encoding = 'utf-8')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,19,20,21,24) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
final = pd.concat([data[['uid', 'date', 'time']], pd.get_dummies(data['class'])], axis=1)
final

,uid,date,time,coupondown_가구,coupondown_간식,coupondown_버거,coupondown_음료,coupondown_음식,coupondown_의류,coupondown_필수품,coupondown_할인쿠폰,coupondown_화장품,couponview_가구,couponview_간식,couponview_건강식품,couponview_버거,couponview_음료,couponview_음식,couponview_의류,couponview_필수품,couponview_할인쿠폰,couponview_화장품,dutumissue_건강식품,dutumissue_쇼핑몰,dutumissue_신발,dutumissue_에뛰드,dutumissue_오케이캐쉬백,dutumissue_옷,dutumissue_유아용품,dutumissue_음식,dutumissue_책,dutumissue_티비,dutumview_가구,dutumview_건강식품,dutumview_쇼핑몰,dutumview_신발,dutumview_에뛰드,dutumview_오케이캐쉬백,dutumview_옷,dutumview_유아용품,dutumview_음식,dutumview_책,dutumview_티비
0,6837732,2019-11-13,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,6837732,2019-08-10,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,8379810,2019-08-11,14,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,8379810,2019-08-11,14,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,8379810,2019-04-19,15,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292279,8484716,2019-04-07,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
292280,8418892,2019-02-19,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
292281,8418892,2019-02-20,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
292282,8418892,2019-02-20,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
data = final.groupby(['uid', 'date'])

In [ ]:
max = -1

for i in data:
  if i[1].shape[0] > max:
    max = i[1].shape[0]

max

253

In [ ]:
lst = []

for i in data:
  lst.append(i[1].shape[0])

In [ ]:
data = np.array(data)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [ ]:
final2 = np.zeros((1, max, 40))

for i in tqdm_notebook(range(data.shape[0])):
  temp = data[i][1].iloc[:, 3:]
  temp = temp[temp.sum(axis=1) != 0]
  if temp.shape[0] < 6:
    continue
  if temp.shape[0] < max:
    temp2 = np.pad(temp.values, ((0, max-temp.shape[0]), (0, 0)), 'constant', constant_values=(0))
    temp2 = temp2.reshape(1, max, 40)
  else:
    temp2 = np.array(temp.iloc[:max, :]).reshape(1, max, 40)
  final2 = np.vstack([final2, temp2])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [ ]:
final2 = final2[1:, :, :]
final2.shape

(9930, 253, 40)

In [ ]:
np.save('/content/drive/Shared drives/SK플래닛_OCB/raw_data/동신&용준/final_253_6.npy', final2)

In [ ]:
final2 = np.load('/content/drive/Shared drives/SK플래닛_OCB/raw_data/동신&용준/final_253_6.npy')

# P_score 만들기

In [ ]:
pscore = final.pivot_table(index='uid', columns=['action', 'category'], aggfunc='count', values='date').fillna(0)
pscore

action   coupondown                            ... dutumview                         
category         가구   간식    버거   음료   음식   의류  ...    오케이캐쉬백    옷 유아용품   음식    책   티비
uid                                            ...                                   
358             0.0  0.0   0.0  2.0  0.0  0.0  ...       1.0  0.0  0.0  0.0  0.0  0.0
943             0.0  0.0   6.0  0.0  0.0  0.0  ...       0.0  0.0  0.0  0.0  0.0  0.0
3405            0.0  1.0  10.0  0.0  0.0  0.0  ...       0.0  0.0  0.0  0.0  0.0  0.0
4151            0.0  0.0   1.0  0.0  0.0  0.0  ...       0.0  0.0  0.0  0.0  0.0  0.0
4405            0.0  0.0   0.0  0.0  0.0  0.0  ...       0.0  0.0  0.0  0.0  0.0  0.0
...             ...  ...   ...  ...  ...  ...  ...       ...  ...  ...  ...  ...  ...
8519108         0.0  6.0   6.0  0.0  3.0  0.0  ...       0.0  0.0  0.0  0.0  0.0  0.0
8523595         0.0  0.0   0.0  0.0  0.0  0.0  ...       0.0  0.0  0.0  0.0  0.0  0.0
8523978         0.0  5.0   7.0  3.0  2.0  0.0  ...       0.0  0.0  0.0  0.0  0.0  0.0
8525646         0.0  0.0   5.0  0.0  0.0  0.0  ...       0.0  0.0  0.0  0.0  0.0  0.0
8528333         0.0  0.0   2.0  0.0  0.0  0.0  ...       2.0  0.0  0.0  0.0  0.0  0.0

[11550 rows x 40 columns]

In [ ]:
for i in range(pscore.shape[0]):
  count = 0
  summation = 0
  for j in range(pscore.shape[1]):
    if pscore.iloc[i, j] > 0:
      count += 1
      summation += pscore.iloc[i, j]
  pscore.iloc[i,:] = pscore.iloc[i,:] / (summation / count)

In [ ]:
pscore

action   coupondown                                     ... dutumview                    
category         가구        간식        버거   음료        음식  ...         옷 유아용품   음식    책   티비
uid                                                     ...                              
358             0.0  0.000000  0.000000  1.2  0.000000  ...       0.0  0.0  0.0  0.0  0.0
943             0.0  0.000000  1.384615  0.0  0.000000  ...       0.0  0.0  0.0  0.0  0.0
3405            0.0  0.160000  1.600000  0.0  0.000000  ...       0.0  0.0  0.0  0.0  0.0
4151            0.0  0.000000  1.000000  0.0  0.000000  ...       0.0  0.0  0.0  0.0  0.0
4405            0.0  0.000000  0.000000  0.0  0.000000  ...       0.0  0.0  0.0  0.0  0.0
...             ...       ...       ...  ...       ...  ...       ...  ...  ...  ...  ...
8519108         0.0  1.826087  1.826087  0.0  0.913043  ...       0.0  0.0  0.0  0.0  0.0
8523595         0.0  0.000000  0.000000  0.0  0.000000  ...       0.0  0.0  0.0  0.0  0.0
8523978         0.0  1.500000  2.100000  0.9  0.600000  ...       0.0  0.0  0.0  0.0  0.0
8525646         0.0  0.000000  1.000000  0.0  0.000000  ...       0.0  0.0  0.0  0.0  0.0
8528333         0.0  0.000000  1.333333  0.0  0.000000  ...       0.0  0.0  0.0  0.0  0.0

[11550 rows x 40 columns]

# SGD를 활용한 Matrix Factorization

In [ ]:
class MF():
    def __init__(self, ratings, K, alpha, beta, iteration, verbose=True):
        self.R = np.array(ratings)                        # 원행렬
        self.num_users, self.num_items = np.shape(self.R) # 유저, 아이템 수
        self.K = K                                        # 잠재요인 수
        self.alpha = alpha                                # 학습률
        self.beta = beta                                  # 정규화 계수
        self.iteration = iteration                        # 반복횟수
        self.verbose = verbose                            # 중간과정 표시
        
    
    def RMSE(self):
        xs, ys = self.R.nonzero()
        self.predictions = []
        self.errors = []
        for x, y in zip(xs, ys):
            prediction = self.get_prediction(x, y)
            self.predictions.append(prediction)
            self.errors.append(self.R[x, y] - prediction)
        
        # 계산을 위해 numpy array로 변경
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        return np.sqrt(np.mean(self.errors**2))
    
    
    def train(self): 
        # 초기값 설정
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # bias 초기값 설정
        self.b_u = np.zeros(self.num_users)
        self.b_d = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R.nonzero()])

        # 값이 존재하는 data 불러오기
        rows, columns = self.R.nonzero()
        self.samples = [(i, j, self.R[i,j]) for i, j in zip(rows, columns)]

        # SGD 진행
        training_process = []
        for i in range(self.iteration):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse = self.RMSE()
            training_process.append((i+1, rmse))
            if self.verbose:
                if (i+1) % 10 == 0:
                    print("Iteration: %d : Train RMSE = %.4f " % (i+1, rmse))
        return training_process
    
    def get_prediction(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i,:].dot(self.Q[j,:].T)
        return prediction
    
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_prediction(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

In [ ]:
mf = MF(pscore, 40, 0.001, 0.02, 1500)

mf.train()

In [ ]:
print(mf.P.shape, mf.Q.shape)

(11550, 40) (40, 40)


In [ ]:
pscore2 = mf.P.dot(mf.Q.T)
pd.DataFrame(pscore2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0,-0.031013,0.020990,-0.013597,0.275597,-0.062494,-0.039763,-0.066222,-0.007925,-0.042682,-0.030791,0.014223,-0.019674,-0.098596,0.292343,-0.138401,-0.009748,-0.066105,0.002341,0.031593,-0.040070,-0.011079,-0.057893,-0.046641,-0.043940,-0.002635,-0.049171,-0.003081,-0.023427,-0.022291,0.015057,-0.029971,0.010464,-0.115943,0.005521,-0.259823,-0.018027,-0.010197,-0.000678,-0.041326,0.014334
1,-0.068634,0.010771,0.358022,-0.043462,-0.054403,-0.038537,-0.033817,-0.014636,-0.046656,-0.056720,-0.052273,0.037322,0.039658,-0.073477,-0.053415,-0.012803,-0.023638,-0.059461,0.009762,-0.039912,-0.037710,-0.016574,-0.024977,-0.033036,-0.081250,-0.000110,-0.031905,-0.043421,-0.049341,0.027416,-0.172017,-0.068380,-0.071046,-0.023165,-0.051596,-0.044594,0.008335,-0.051081,-0.045399,-0.030505
2,-0.114920,-0.303097,0.541781,-0.074790,-0.125261,-0.158832,-0.133688,-0.147141,-0.129388,-0.124151,-0.384187,-0.286138,0.662980,-0.107902,-0.194272,-0.184605,-0.151038,-0.117214,-0.029180,-0.147796,-0.182284,-0.100581,-0.111027,-0.129568,-0.217062,-0.013030,-0.148131,-0.099301,-0.133535,0.085226,-0.193259,-0.246081,-0.203040,-0.186826,-0.213769,-0.154528,-0.067230,-0.214028,-0.163673,-0.119703
3,0.035194,0.054531,-0.088562,0.115165,0.066833,0.036864,0.019757,0.061788,0.060468,0.055636,0.025435,0.065988,-0.412456,0.088059,0.050636,0.084546,0.035197,0.144115,0.098994,0.038780,0.060872,0.038041,0.035697,0.067057,0.075278,0.019077,0.050656,0.031147,0.043315,-0.035217,0.033548,0.072254,0.060004,0.061639,0.047161,0.065558,0.073894,0.069965,0.046903,0.052502
4,0.151599,0.191097,-0.395849,0.219863,0.190595,0.169401,0.099960,0.169236,0.168030,0.241066,0.121946,0.308840,-0.941238,0.201090,0.215636,0.228932,0.132766,0.160009,0.195063,0.152628,0.240533,0.135447,0.180830,0.171786,0.206395,0.072751,0.162837,0.128252,0.176707,-0.088754,0.186826,0.279016,0.296610,0.276786,0.231352,0.154503,0.211878,0.222853,0.209995,0.185298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11545,-0.152393,0.540607,0.500278,-0.158661,0.001120,-0.017162,0.065829,-0.111091,-0.067870,-0.097322,0.612587,0.145367,0.268176,-0.342309,0.025088,0.043850,0.080759,-0.455466,-0.466019,-0.006968,-0.267187,-0.015027,-0.098682,-0.005850,0.006827,-0.105927,0.018638,-0.074923,-0.099012,-0.007598,-0.027802,-0.368038,0.105181,-0.155507,0.075358,-0.079037,-0.209875,0.069221,-0.021838,-0.145183
11546,0.017900,0.021863,-0.036008,0.067257,0.014259,0.026723,0.014130,0.001870,-0.005967,0.051438,0.033672,0.045972,-0.057995,0.084305,0.030916,0.019665,0.022865,-0.053613,-0.073894,0.010935,0.066525,0.000312,0.014207,-0.006821,-0.021643,-0.016533,0.016341,0.009139,0.025230,-0.002626,0.000246,0.079185,0.024335,0.010474,-0.009821,0.027507,-0.009854,0.012142,0.014842,0.024423
11547,-0.255966,0.281334,0.611139,-0.013635,-0.153406,-0.108848,-0.014117,-0.173993,-0.202507,-0.347225,0.294113,0.019306,0.362968,-0.105960,-0.278664,-0.100030,-0.007552,-0.335579,-0.327474,-0.129641,-0.128178,-0.159791,-0.147616,-0.188824,-0.246575,-0.130668,-0.089466,-0.165783,-0.182002,0.077508,-0.071792,-0.149782,-0.196270,-0.221920,-0.049943,-0.180292,-0.208118,-0.133220,-0.225545,-0.167650
11548,0.042349,-0.013006,-0.099457,0.015151,0.031102,0.018170,-0.002120,0.038978,0.041632,0.078466,-0.017410,0.009173,-0.124100,0.020582,0.048739,0.039071,-0.004657,0.040855,0.116285,0.023806,0.005301,0.033240,0.030792,0.040354,0.021733,0.027996,0.023068,0.033277,0.025067,-0.017623,-0.025760,0.012033,0.052576,0.037247,0.035248,0.043384,0.061717,0.039446,0.064149,0.029146


# One hot encoding embedding

In [ ]:
# Parent Class Layer를 상속받음
class OnehotEmbedding(Layer):

    # 생성자
    # Nembeddings ; emedding 차원
    # **kwargs : keyword arguments -> 입력을 받으면 dictionary형태로 저장
    def __init__(self, Nembeddings, **kwargs):
        self.Nembeddings = Nembeddings
        super(OnehotEmbedding, self).__init__(**kwargs)

    # weights를 생성하는 함수
    def build(self, input_shape):
        # 훈련 가능한 가중치 행렬(embedding matrix) 생성

        # init_val = temp
        init_val = mf.Q
        initializer = tf.constant_initializer(init_val)
        self.kernel = self.add_weight(name='kernel',
                                      shape=(input_shape[2], self.Nembeddings),
                                      initializer='uniform', # uniform, initializer
                                      trainable=True)
        super(OnehotEmbedding, self).build(input_shape) 

    # layer의 연산 부분을 담당
    def call(self, x):
      return K.dot(x, self.kernel)

    # 출력 output shape 계산
    def compute_output_shape(self, input_shape):
      return (input_shape[0], input_shape[1], self.Nembeddings)

# 모델 생성

In [ ]:
model = Sequential()
emb = OnehotEmbedding(40, input_shape=(6,40))
model.add(emb)
model.add(LSTM(512, input_shape = (6,40), return_sequences=True))
model.add(LSTM(256, input_shape = (6,40), return_sequences= True))
model.add(LSTM(128, input_shape = (6,40), return_sequences= True))
model.add(LSTM(64, input_shape = (6,40), return_sequences= True))
model.add(Flatten())
model.add(Dense(40, activation='softmax'))

model.compile(optimizer= Adam(),
              loss=keras.losses.kullback_leibler_divergence,
              metrics=['acc'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
onehot_embedding_1 (OnehotEm (None, 6, 40)             1600      
_________________________________________________________________
lstm_8 (LSTM)                (None, 6, 512)            1132544   
_________________________________________________________________
lstm_9 (LSTM)                (None, 6, 256)            787456    
_________________________________________________________________
lstm_10 (LSTM)               (None, 6, 128)            197120    
_________________________________________________________________
lstm_11 (LSTM)               (None, 6, 64)             49408     
_________________________________________________________________
flatten_2 (Flatten)          (None, 384)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 40)               

In [ ]:
model2 = Sequential()
model2.add(LSTM(512, input_shape = (6,40), return_sequences=True))
model2.add(LSTM(256, input_shape = (6,40), return_sequences=True))
model2.add(LSTM(128, input_shape = (6,40), return_sequences=True))
model2.add(LSTM(64, input_shape = (6,40), return_sequences=True))
model2.add(Flatten())
model2.add(Dense(40, activation = 'softmax'))

model2.compile(optimizer= Adam(),
              loss=keras.losses.kullback_leibler_divergence,
              metrics=['acc'])

model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 6, 512)            1132544   
_________________________________________________________________
lstm_5 (LSTM)                (None, 6, 256)            787456    
_________________________________________________________________
lstm_6 (LSTM)                (None, 6, 128)            197120    
_________________________________________________________________
lstm_7 (LSTM)                (None, 6, 64)             49408     
_________________________________________________________________
flatten_1 (Flatten)          (None, 384)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 40)                15400     
Total params: 2,181,928
Trainable params: 2,181,928
Non-trainable params: 0
____________________________________________

In [ ]:
model3 = Sequential()
emb = OnehotEmbedding(40, input_shape=(6,40))
model3.add(emb)
model3.add(LSTM(512, input_shape = (6,40), return_sequences=True))
model3.add(LSTM(256, input_shape = (6,40), return_sequences= True))
model3.add(LSTM(128, input_shape = (6,40), return_sequences= True))
model3.add(LSTM(64, input_shape = (6,40), return_sequences= True))
model3.add(Flatten())
model3.add(Dense(40, activation='softmax'))

model3.compile(optimizer= Adam(),
              loss=keras.losses.kullback_leibler_divergence,
              metrics=['acc'])

model3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
onehot_embedding_2 (OnehotEm (None, 6, 40)             1600      
_________________________________________________________________
lstm_12 (LSTM)               (None, 6, 512)            1132544   
_________________________________________________________________
lstm_13 (LSTM)               (None, 6, 256)            787456    
_________________________________________________________________
lstm_14 (LSTM)               (None, 6, 128)            197120    
_________________________________________________________________
lstm_15 (LSTM)               (None, 6, 64)             49408     
_________________________________________________________________
flatten_3 (Flatten)          (None, 384)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 40)               

# train, test split

In [ ]:
idx = np.random.choice(final2.shape[0], int(final2.shape[0]*0.8), replace=False)

train, test = [], []

for i in tqdm_notebook(range(final2.shape[0])):
  if i in idx:
    train.append(i)
  else:
    test.append(i)
    
print(len(train), len(test))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """



7944 1986


In [ ]:
train = final2[train]
test = final2[test]

print(train.shape, test.shape)

(7944, 253, 40) (1986, 253, 40)


# 학습

In [ ]:
# 순차적으로 timestamp를 늘려가면서 학습 
for i in range(246): # 246
  print(i, '번째 진행중')

  x_data = train[:, i:(6+i), :]
  y_data = train[:, (6+i), :]

  lst = []
  for i in range(y_data.shape[0]):
    if y_data[i].sum() != 0:
      lst.append(i)

  x_data = x_data[lst]
  y_data = y_data[lst]

  num_epoch = 0
  if x_data.shape[0] >= 50:
    num_epoch = 30
  elif x_data.shape[0] >= 30:
    num_epoch = 20
  else:
    num_epoch = 10
  history = model.fit(x_data, y_data,
                      epochs=num_epoch,
                      batch_size=128)

In [ ]:
for i in range(246):
  print(i, '번째 진행중')

  x_data = train[:, i:(6+i), :]
  y_data = train[:, (6+i), :]

  lst = []
  for i in range(y_data.shape[0]):
    if y_data[i].sum() != 0:
      lst.append(i)

  x_data = x_data[lst]
  y_data = y_data[lst]

  num_epoch = 0
  if x_data.shape[0] >= 50:
    num_epoch = 30
  elif x_data.shape[0] >= 30:
    num_epoch = 20
  else:
    num_epoch = 10
  history = model2.fit(x_data, y_data,
                       epochs=num_epoch,
                       batch_size=128)

In [ ]:
for i in range(246): # 246
  print(i, '번째 진행중')

  x_data = train[:, i:(6+i), :]
  y_data = train[:, (6+i), :]

  lst = []
  for i in range(y_data.shape[0]):
    if y_data[i].sum() != 0:
      lst.append(i)

  x_data = x_data[lst]
  y_data = y_data[lst]

  num_epoch = 0
  if x_data.shape[0] >= 50:
    num_epoch = 30
  elif x_data.shape[0] >= 30:
    num_epoch = 20
  else:
    num_epoch = 10
  history = model3.fit(x_data, y_data,
                       epochs=num_epoch,
                       batch_size=128)

# 상관관계 확인 (관련있는 item 찾기)

In [ ]:
cor = pd.DataFrame(pscore).corr()
cor

action              coupondown            ... dutumview          
category                    가구        간식  ...         책        티비
action     category                       ...                    
coupondown 가구         1.000000  0.067939  ... -0.000374 -0.005611
           간식         0.067939  1.000000  ...  0.035917 -0.008115
           버거         0.012749  0.313401  ... -0.007974 -0.019493
           음료         0.046485  0.294188  ...  0.025277  0.010996
           음식        -0.000061  0.185324  ...  0.001457  0.006244
           의류        -0.002422  0.006023  ... -0.001794  0.016540
           필수품        0.058420  0.031461  ... -0.001095 -0.002074
           할인쿠폰       0.065604  0.071825  ...  0.017579  0.018970
           화장품        0.021716  0.040148  ...  0.002505  0.009844
couponview 가구         0.122617 -0.024934  ...  0.000377  0.011304
           간식         0.050290  0.715157  ...  0.032095 -0.003045
           건강식품       0.001498 -0.014507  ... -0.003477  0.006884
           버거         0.006734  0.208817  ... -0.000394 -0.006861
           음료         0.029690  0.189506  ...  0.020045  0.025530
           음식        -0.002749  0.099135  ...  0.004608  0.004417
           의류        -0.005259  0.008668  ... -0.004176  0.045645
           필수품        0.075337  0.031799  ... -0.001410  0.000347
           할인쿠폰       0.008156 -0.056925  ...  0.015582  0.087657
           화장품        0.010134 -0.054049  ...  0.003880  0.082910
dutumissue 건강식품       0.009701  0.019551  ...  0.019879  0.043653
           쇼핑몰        0.007529  0.010944  ...  0.064772  0.014582
           신발        -0.004290 -0.001633  ... -0.003565 -0.005560
           에뛰드        0.014615  0.002073  ...  0.001464 -0.000192
           오케이캐쉬백     0.010170 -0.020932  ... -0.000085  0.044126
           옷          0.004497  0.001776  ...  0.012192  0.006021
           유아용품      -0.001290 -0.009153  ... -0.000933  0.014354
           음식        -0.002692  0.021450  ...  0.002164  0.003811
           책          0.000071  0.043360  ...  0.852989  0.034599
           티비        -0.001421  0.006926  ...  0.093608  0.508633
dutumview  가구        -0.000621 -0.003395  ... -0.000460 -0.000870
           건강식품       0.007898 -0.032525  ...  0.010746  0.060530
           쇼핑몰        0.006805  0.002937  ...  0.085929  0.023639
           신발        -0.004336 -0.010731  ... -0.003897  0.002117
           에뛰드        0.014998  0.003408  ...  0.017761  0.000020
           오케이캐쉬백     0.004936 -0.035071  ...  0.014489  0.110981
           옷         -0.006653 -0.021299  ...  0.034601  0.039056
           유아용품      -0.001447 -0.010331  ... -0.000682  0.021838
           음식        -0.002996  0.010489  ...  0.000480  0.001459
           책         -0.000374  0.035917  ...  1.000000  0.033856
           티비        -0.005611 -0.008115  ...  0.033856  1.000000

[40 rows x 40 columns]

# precision@5, 7, 10, // AP@5, 7, 10 평가

In [ ]:
cor = pd.DataFrame(pscore.values).corr()

dic = {}

for i in cor.columns:
  dic[i] = cor.iloc[:, i].sort_values(ascending=False).head(11).index[1:].tolist()

dic

{0: [9, 16, 1, 7, 6, 10, 3, 13, 8, 33],
 1: [10, 2, 3, 12, 13, 4, 14, 7, 0, 27],
 2: [12, 1, 4, 10, 3, 14, 13, 8, 7, 16],
 3: [13, 1, 10, 2, 4, 12, 7, 14, 0, 16],
 4: [14, 2, 12, 1, 10, 3, 13, 26, 37, 8],
 5: [15, 24, 39, 35, 13, 10, 3, 17, 1, 12],
 6: [16, 0, 3, 2, 1, 10, 13, 12, 9, 4],
 7: [17, 3, 1, 8, 0, 22, 13, 30, 18, 10],
 8: [18, 17, 7, 30, 35, 23, 20, 19, 2, 33],
 9: [11, 0, 35, 34, 17, 13, 8, 16, 19, 7],
 10: [1, 12, 13, 2, 3, 14, 4, 0, 7, 27],
 11: [9, 28, 19, 35, 24, 20, 39, 23, 16, 32],
 12: [2, 10, 14, 1, 4, 13, 3, 16, 6, 26],
 13: [3, 10, 1, 12, 14, 4, 2, 17, 7, 30],
 14: [4, 12, 10, 2, 13, 1, 26, 3, 37, 28],
 15: [5, 35, 39, 17, 10, 13, 24, 8, 18, 1],
 16: [6, 0, 13, 3, 12, 2, 1, 10, 9, 7],
 17: [18, 30, 7, 34, 35, 39, 8, 13, 19, 9],
 18: [17, 30, 8, 34, 39, 35, 7, 19, 13, 31],
 19: [30, 23, 34, 22, 20, 28, 17, 33, 24, 35],
 20: [31, 24, 35, 21, 19, 38, 27, 32, 33, 8],
 21: [32, 20, 31, 35, 24, 19, 30, 34, 7, 33],
 22: [33, 24, 19, 7, 20, 35, 31, 23, 37, 26],
 23: [34, 

In [ ]:
final = {}

for num in [5, 7, 10]:
  result = []
  result2 = []
  for i in range(246): # 44
    print(i, '번째 진행중')

    x_data = test[:, i:(6+i), :]
    y_data = test[: , (6+i), :]

    lst = []
    for j in range(y_data.shape[0]):
      if y_data[j].sum() != 0:
        lst.append(j)

    x_data = x_data[lst]
    y_data = y_data[lst]

    for k in range(x_data.shape[0]):
      # 예측
      y_pred = model.predict(x_data[k].reshape(-1, 6, 40))

      # 실제
      y_test = np.where(y_data[k] == 1)[0][0]
      temp = pd.DataFrame(y_pred).T
      temp.columns = ['prob']

      # 예측 값 상위 N개 추출
      y_pred_idx = temp.sort_values(by='prob', ascending=False).head(num).index

      cnt = 0
      lst2 = []
      flag = False
      for idx,value in enumerate(dic[y_test][:num]):
        if value in y_pred_idx:
          flag = True
          cnt += 1
        lst2.append(cnt / (idx+1))

      if flag:
        result.append(1)
      else:
        result.append(0)

      result2.append(sum(lst2) / len(lst2))
        

  final[num] = [sum(result) / len(result), sum(result2) / len(result2)]
      
final
# {5: [0.9164382546855323, 0.5398782625873361],
#  7: [0.9381435661900882, 0.7261045791751416],
#  10: [0.9885176679754056, 0.8348675974822901]}

In [ ]:
final2 = {}

for num in [5, 7, 10]:
  result = []
  result2 = []
  for i in range(246): # 44
    print(i, '번째 진행중')

    x_data = test[:, i:(6+i), :]
    y_data = test[: , (6+i), :]

    lst = []
    for j in range(y_data.shape[0]):
      if y_data[j].sum() != 0:
        lst.append(j)

    x_data = x_data[lst]
    y_data = y_data[lst]

    for k in range(x_data.shape[0]):
      # 예측
      y_pred = model2.predict(x_data[k].reshape(-1, 6, 40))

      # 실제
      y_test = np.where(y_data[k] == 1)[0][0]
      temp = pd.DataFrame(y_pred).T
      temp.columns = ['prob']

      # 예측 값 상위 N개 추출
      y_pred_idx = temp.sort_values(by='prob', ascending=False).head(num).index

      cnt = 0
      lst2 = []
      flag = False
      for idx,value in enumerate(dic[y_test][:num]):
        if value in y_pred_idx:
          flag = True
          cnt += 1
        lst2.append(cnt / (idx+1))

      if flag:
        result.append(1)
      else:
        result.append(0)

      result2.append(sum(lst2) / len(lst2))
        

  final2[num] = [sum(result) / len(result), sum(result2) / len(result2)]
      
final2

# {5: [0.782216460478554, 0.4722816504926111],
#  7: [0.8247359063634343, 0.6718370672959558],
#  10: [0.8682203126157493, 0.7611552648815402]}

In [ ]:
final3 = {}

for num in [5, 7, 10]:
  result = []
  result2 = []
  for i in range(246): # 44
    print(i, '번째 진행중')

    x_data = test[:, i:(6+i), :]
    y_data = test[: , (6+i), :]

    lst = []
    for j in range(y_data.shape[0]):
      if y_data[j].sum() != 0:
        lst.append(j)

    x_data = x_data[lst]
    y_data = y_data[lst]

    for k in range(x_data.shape[0]):
      # 예측
      y_pred = model3.predict(x_data[k].reshape(-1, 6, 40))

      # 실제
      y_test = np.where(y_data[k] == 1)[0][0]
      temp = pd.DataFrame(y_pred).T
      temp.columns = ['prob']

      # 예측 값 상위 N개 추출
      y_pred_idx = temp.sort_values(by='prob', ascending=False).head(num).index

      cnt = 0
      lst2 = []
      flag = False
      for idx,value in enumerate(dic[y_test][:num]):
        if value in y_pred_idx:
          flag = True
          cnt += 1
        lst2.append(cnt / (idx+1))

      if flag:
        result.append(1)
      else:
        result.append(0)

      result2.append(sum(lst2) / len(lst2))
        

  final3[num] = [sum(result) / len(result), sum(result2) / len(result2)]
      
final3

# {5: [0.7974005481887547, 0.5327932933303341],
#  7: [0.8311800874138825, 0.6673415465897099],
#  10: [0.8968130972664641, 0.8068418266138861]}